In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import os
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Utilice el dataset New York City Taxi Trip Duration para predecir la columna trip_duration (Duración del viaje).

1. Separe el dataset en Train y Test (1 pt)
2. Realice Análisis de los datos (EDA) con gráficos o tablas (4 pts)
3. Cual es la distribución de los datos (2 pt)
4. Grafique la matriz de correlación (2 pt)
5. Ajuste un modelo con scikitlearn para realizar la predicción (2 pts)
6. Muestre sus resultados (¡Sea creativo!).

### 0. Se importa el dataset

In [ ]:
from zipfile import ZipFile

with ZipFile('/kaggle/input/nyc-taxi-trip-duration/train.zip','r') as file:
    file.extractall()
with ZipFile('/kaggle/input/nyc-taxi-trip-duration/test.zip','r') as file:
    file.extractall()  
with ZipFile('/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip','r') as file:
    file.extractall()

In [ ]:
train_df=pd.read_csv("./train.csv")
test_df=pd.read_csv("./test.csv")

Para efectos de la tarea, se usara un dataset reducido mediante una muestra del dataset original del 10%

In [ ]:
train_df=train_df.sample(frac=0.1, replace=True, random_state=1)
test_df2=test_df.sample(frac=0.1, replace=True, random_state=1)

In [ ]:
train_df.shape

In [ ]:
#Se revisa que no haya valores faltantes para ver si es necesario darle un tratamiento
train_df.isna().sum().sort_values()

### 1. Análisis exploratorio de los datos

Para efectos prácticos, se realizará primero el EDA. El análisis se hará sobre el dataset de training(train_df completo), pero si se deben realizar cambios se van a generar sobre ambos datasets. Primero se van a validar los tipos de datos que se tienen en las columnas, para analizar si hay variables categoricas que necesiten ser transformadas. Despues se correran una serie de estadisticas bàsicas que nos pueden indicar si hay outliers o bien si las dimensiones de las features son muy distintas entre si

In [ ]:
train_df.dtypes

In [ ]:
test_df.dtypes

In [ ]:
# Se revisan las locaciones de dropoff y pickup
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
sns.scatterplot(train_df['pickup_longitude'],train_df['pickup_latitude'],ax=ax1)
sns.scatterplot(train_df['dropoff_longitude'],train_df['dropoff_latitude'],ax=ax2)

Como estas features son continuas,se obtendra la distancia entre ambos puntos

In [ ]:
# concatenating lat and long to create a consolidated location as accepted by havesine function
train_df['PickCoor'] = list(zip(train_df.pickup_latitude,train_df.pickup_longitude))
train_df['DropCoor'] = list(zip(train_df.dropoff_latitude,train_df.dropoff_longitude))

In [ ]:
import haversine as hs
def distance_from(loc1,loc2): 
    dist=hs.haversine(loc1,loc2)
    return round(dist,2)


In [ ]:
for _,row in train_df.iterrows():
    train_df['distance']=train_df['PickCoor'].apply(lambda x: distance_from(row.DropCoor,x))

In [ ]:
train_df

In [ ]:
train_df.drop(['pickup_longitude','pickup_latitude','dropoff_latitude','dropoff_longitude'],1,inplace=True)
test_df.drop(['pickup_longitude','pickup_latitude','dropoff_latitude','dropoff_longitude'],1,inplace=True)

In [ ]:
#se transforman las fechas guardadas como objetos a el tipo datetime. Se hace para los dos datasets
train_df['pickup_datetime'] = pd.to_datetime(train_df.pickup_datetime)
test_df['pickup_datetime'] = pd.to_datetime(test_df.pickup_datetime)

#Se elimina la fecha de dropout que no esta en dataset test
train_df.drop(['dropoff_datetime'],1,inplace=True)

In [ ]:
train_df.describe()

Con este cuadro podemos observar varias situaciones que requieren tratamiento:
1. Se eliminaran las filas en donde la cantidad de pasajeros es 0
2. Se eliminaran Outliers especialmente en la columna de trip duration, que es la que un STD más grande
3. Distancias muy pequeñas o muy grandes


In [ ]:
#Se imprime la cantidad de rows para comprobar que se borran las filas en cada paso.
train_df.shape

In [ ]:
#Elimino las filas con 0 pasajeros

train_df = train_df[(train_df.passenger_count > 0)]
test_df = test_df[(test_df.passenger_count > 0)]

In [ ]:
#elimino las distancias que sean menores que 1km y mayores a 10 km
train_df = train_df[(train_df.distance > 1)]
train_df = train_df[(train_df.distance < 10)]

In [ ]:
#elimino outliers en el tiempo, para esto se usará Z score
train_df['z_scores'] = pd.DataFrame(stats.zscore(train_df.trip_duration))
train_df.describe()

In [ ]:
axes = plt.axes()
axes.set_xlim([-0, 1.5])
sns.boxplot(x=train_df["z_scores"])

Se define como limite para recorte un valor de z_score de 0.35, que es lo que nos muestra el boxplot como límite

In [ ]:
train_df.drop(train_df[train_df['z_scores'] > 0.35].index, inplace = True)
train_df=train_df.drop("z_scores",axis=1)

Se obtiene un dataframe con un recorte debido a que esas filas son consideradas outliers

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

Todavía hay valores de tiempo muy grandes, de casi 24 horas por lo que se considera que es un error. Se definira otro límite como viajes de máximo 1,5 horas , es decir 5400 minutos

In [ ]:
train_df = train_df[(train_df.trip_duration < 5400)]

Se grafica el dataset para ver su comportamiento con estos cambios

In [ ]:
train_df.hist(bins=100, figsize=(15,15))
plt.show()

De los graficos anteriores, se puede observar que aún con la eliminación de los outliers no se puede ver una distribución clara para la etiqueta y para la distancia de nuestro dataset, por lo que se transformaran. La etiqueta esta sesgada hacia la izquierda


### 2. Distribución de los datos y transformaciones realizadas

Se aplicará una transformación logaritmica a la etiqueta de duracion de tiempo y distancia, posteriormente se normalizaran las features y se hara one hot encoder.


In [ ]:
plt.subplots(figsize=(18,6))
train_df['trip_duration'] = np.log(train_df['trip_duration'].values)
train_df['distance'] = np.log(train_df['distance'].values)
plt.hist(train_df['trip_duration'].values, bins=100)
plt.xlabel('log(trip_duration)')
plt.ylabel('registros')
plt.show()

In [ ]:
train_df.describe()

Para el modelo que se utilizara no se puede usar la fecha como un feature, por lo que se tranformara en tres otras variables como lo son: mes, dia de la semana y hora del viaje

In [ ]:
#Se transforma la columna de fecha en:mes, dia de la semana y hora ya que es lo que se puede procesar en una regresion

train_df['month'] = train_df.pickup_datetime.dt.month
train_df['weekday'] = train_df.pickup_datetime.dt.weekday
train_df['hour'] = train_df.pickup_datetime.dt.hour
train_df.drop(['pickup_datetime'], axis=1, inplace=True)

test_df['month'] = test_df.pickup_datetime.dt.month
test_df['weekday'] = test_df.pickup_datetime.dt.weekday
test_df['hour'] = test_df.pickup_datetime.dt.hour
test_df.drop(['pickup_datetime'], axis=1, inplace=True)

Se realiza one hot encoding a las variables categóricas, para poder ser consideradas dentro del modelo

In [ ]:
# Se realiza el one hot encoding a las variables categoricas

train_df = pd.concat([train_df, pd.get_dummies(train_df['vendor_id'])], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df['vendor_id'])], axis=1)
train_df.drop(['vendor_id'], axis=1, inplace=True)
test_df.drop(['vendor_id'], axis=1, inplace=True)

train_df = pd.concat([train_df, pd.get_dummies(train_df['store_and_fwd_flag'])], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df['store_and_fwd_flag'])], axis=1)
train_df.drop(['store_and_fwd_flag'], axis=1, inplace=True)
test_df.drop(['store_and_fwd_flag'], axis=1, inplace=True)



In [ ]:
train_df.head(3)

### 3. Matriz de correlación

In [ ]:
correlacion=train_df.corr()
plt.figure(figsize = (15,10))
sns.heatmap(correlacion, annot = True)
plt.show()

Del grafico anterior, se puede observar que las features en general tienen una correlación lineal debil con la etiqueta. La correlacion lineal mas llamativa es la del conductor con la cantidad de pasajeros, pero esto puede ser relacionado a que uno de ellos tiene un taxi con mayor capacidad por ejemplo.

### 4. División del dataset y modelado

Con los cambios realizados en los dataset, se procede a realizar la divisón del train_df en dos datasets, uno de entrenamiento y otro de prueba

In [ ]:
features = train_df.drop(["id","PickCoor","DropCoor"],axis=1)
features=features.drop("trip_duration",axis=1)
labels = train_df["trip_duration"]
prueba=train_df.loc[:,["distance","hour"]]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,labels, test_size=0.30, random_state=42)

In [ ]:
print("Tamaño dataset entranamiento features",X_train.shape,"Tamaño dataset entranamiento labels",y_train.shape)

### 5. Modelado y predicción

Se realizara el modelo mediante una regresion lineal multiple

In [ ]:
#Se utilizara una regresion lineal multiple.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lrmodel = LinearRegression()
lrmodel.fit(X_train, y_train)

In [ ]:
#Se imprimen los coeficientes
b_i = lrmodel.coef_
b_0 = lrmodel.intercept_

print("b0:",b_0, "bi:", b_i)

In [ ]:
# Se realiza la prediccion con la particion de test
predictionTrain=lrmodel.predict(X_train)
predictionTest=lrmodel.predict(X_test)

In [ ]:
#Se calculan los RMSE de ambas predicciones

rmse_train=mean_squared_error(y_train, predictionTrain,squared=False)
rmse_test=mean_squared_error(y_test, predictionTest,squared=False)

print("El RMSE del training fue: ",rmse_train)
print("El RMSE del test fue: ",rmse_test)


Se calcula el R2 

In [ ]:
from sklearn.metrics import r2_score

R2Train=r2_score(y_train, predictionTrain)
R2Test=r2_score(y_test, predictionTest)

print("El R2 del train fue :",R2Train, " y el R2 del test fue: ",R2Test)

In [ ]:
df2 =X_test
df2['y_test']=y_test
df2['y_predicted']=predictionTest

df2.head(10)

Se obtiene una pesima predicción, independientemente se si hace una regresión lineal simple o multiple (con 2 features o con todas); en este caso se llega a dos conclusiones que el modelo escogido no es el más adecuado para realizar la predicción y que posiblemente hacen falta predictores o features que ayuden a explicar el comportamiento de la duración del viaje.

### 6. Resultados

Se grafican los primeros 10 registros, para ver las diferencias más notorias de una manera rápida. De acá se puede observar que al escoger una regresión lineal los valores predecidos estan siempre muy cercanos al valor del intercepto , considerando también que los coeficientes obtenidos son muy bajos.

Ante esto se obtienen predicciones de tiempo muy similares para todos los casos, en un promedio de 6.35

In [ ]:
plt.scatter(x=df2.index[0:10],y=df2.y_test[0:10])
plt.scatter(x=df2.index[0:10],y=df2.y_predicted[0:10])

Se grafica ahora todos los registros y se puede observar lo antes mencionado, las predicciones siempre estan cercanas a un mismo valor o rango, mientras que el dato real es muy variado, lo que hace que sea una predicción érronea con bajos valores de R2 como antes se menciono.

In [ ]:
plt.scatter(x=df2.index,y=df2.y_test)
plt.scatter(x=df2.index,y=df2.y_predicted)